In [11]:
import tensorflow as tf
import numpy as np
import pathlib

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).with_suffix('')
#import image set correctly


batch_size = 32
img_height, img_width = 180, 180
#batch size splits dataset, each image is 180x180x3 - 3 for RGB

train_ds = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.2, subset="training",
  seed=123, image_size=(img_height, img_width),batch_size=batch_size)

class_names = train_ds.class_names

val_ds = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.2, subset="validation",
  seed=123, image_size=(img_height, img_width),batch_size=batch_size)
#split data into training set(0.8) and validation set(0.2)

norm_layer = tf.keras.layers.Rescaling(1./255)
#preprocessing layer to turn [0,255] into [0,1]

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
#to allow for efficient I/O to GPU

num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape = (img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding = 'same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding = 'same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding = 'same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])
# Sequential means passes linearly through. First detect neighbourhood patterns in increasing detail wiht conv,
# then flatten to 1d and use dense to connect to find global features. Output with 5 classes - flower types

model.compile(optimizer = 'adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])
#define optimization program (i.e. gradient descent), loss function and metrics is for reviewing training later

epochs = 13
history = model.fit(train_ds, validation_data = val_ds, epochs = epochs)
#trains the model, fits training data to validation data and repeats 10 times.

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Epoch 1/13
92/92 [==============================] - 7s 69ms/step - loss: 1.4890 - accuracy: 0.3743 - val_loss: 1.1502 - val_accuracy: 0.5068
Epoch 2/13
92/92 [==============================] - 6s 61ms/step - loss: 1.0984 - accuracy: 0.5593 - val_loss: 1.0856 - val_accuracy: 0.5749
Epoch 3/13
92/92 [==============================] - 6s 64ms/step - loss: 1.0195 - accuracy: 0.6121 - val_loss: 1.1046 - val_accuracy: 0.6076
Epoch 4/13
92/92 [==============================] - 6s 65ms/step - loss: 1.0528 - accuracy: 0.6410 - val_loss: 1.4436 - val_accuracy: 0.5272
Epoch 5/13
92/92 [==============================] - 6s 62ms/step - loss: 0.9278 - accuracy: 0.7047 - val_loss: 1.9814 - val_accuracy: 0.5395
Epoch 6/13
92/92 [==============================] - 6s 62ms/step - loss: 3.1545 - accuracy: 0.5760 - val_loss: 2.6349 - val_accuracy: 0.5082
Epoch 7/1

In [49]:
import os
url = 'https://static.scientificamerican.com/sciam/cache/file/A96AF6D1-484D-412F-8E43F6D8882F32AA_source.jpg'
image = tf.keras.utils.load_img(tf.keras.utils.get_file('/Users/joeladams/Documents/Python/Machine Learning/temp.jpg', url), target_size = (180,180))
image.show()

img_array = tf.keras.utils.img_to_array(image)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

for i in range(5):
    print('{}: {:.2f}'.format(class_names[i], score[i]))

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)
os.remove('/Users/joeladams/Documents/Python/Machine Learning/temp.jpg')



1/1 [==============================] - 0s 16ms/step
daisy: 0.00
dandelion: 0.00
roses: 1.00
sunflowers: 0.00
tulips: 0.00
This image most likely belongs to roses with a 100.00 percent confidence.
